In [ ]:
import pandas as pd

!pip install stocksymbol -q

from google.colab import drive
drive.mount('drive')


!pip install gnews -q


## ML
! pip install transformers

!pip install datasets -q
!pip install optuna -q

Mounted at drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.8/115.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.1/517.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.2.0 requires requests>=2.27.1, but you have requests 2.26.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.26.0 which is incompatible.
tweepy 4.14.0 requires requests<3,>=2.27.0, but you have requests 2.26.0 which is i

## Parsing all cycle

### Classical parser


In [ ]:
!pip install dateparser -q

## Source https://github.com/kotartemiy/pygooglenews/blob/master/pygooglenews/__init__.py
import feedparser
from bs4 import BeautifulSoup
import urllib
from dateparser import parse as parse_date
import requests



class GoogleNews:
    def __init__(self, lang = 'en', country = 'USA'):
        self.lang = lang.lower()
        self.country = country.upper()
        self.BASE_URL = 'https://news.google.com/rss'

    def __top_news_parser(self, text):
        """Return subarticles from the main and topic feeds"""
        try:
            bs4_html = BeautifulSoup(text, "html.parser")
            # find all li tags
            lis = bs4_html.find_all('li')
            sub_articles = []
            for li in lis:
                try:
                    sub_articles.append({"url": li.a['href'],
                                         "title": li.a.text,
                                         "publisher": li.font.text})
                except:
                    pass
            return sub_articles
        except:
            return text

    def __ceid(self):
        """Compile correct country-lang parameters for Google News RSS URL"""
        return '?ceid={}:{}&hl={}&gl={}'.format(self.country,self.lang,self.lang,self.country)

    def __add_sub_articles(self, entries):
        for i, val in enumerate(entries):
            if 'summary' in entries[i].keys():
                entries[i]['sub_articles'] = self.__top_news_parser(entries[i]['summary'])
            else:
                entries[i]['sub_articles'] = None
        return entries

    def __scaping_bee_request(self, api_key, url):
        response = requests.get(
            url="https://app.scrapingbee.com/api/v1/",
            params={
                "api_key": api_key,
                "url": url,
                "render_js": "false"
            }
        )
        if response.status_code == 200:
            return response
        if response.status_code != 200:
            raise Exception("ScrapingBee status_code: "  + str(response.status_code) + " " + response.text)

    def __parse_feed(self, feed_url, proxies=None, scraping_bee = None):

        if scraping_bee and proxies:
            raise Exception("Pick either ScrapingBee or proxies. Not both!")

        if proxies:
            r = requests.get(feed_url, proxies = proxies)
        else:
            r = requests.get(feed_url)

        if scraping_bee:
            r = self.__scaping_bee_request(url = feed_url, api_key = scraping_bee)
        else:
            r = requests.get(feed_url)


        if 'https://news.google.com/rss/unsupported' in r.url:
            raise Exception('This feed is not available')

        d = feedparser.parse(r.text)

        if not scraping_bee and not proxies and len(d['entries']) == 0:
            d = feedparser.parse(feed_url)

        return dict((k, d[k]) for k in ('feed', 'entries'))

    def __search_helper(self, query):
        return urllib.parse.quote_plus(query)

    def __from_to_helper(self, validate=None):
        try:
            validate = parse_date(validate).strftime('%Y-%m-%d')
            return str(validate)
        except:
            raise Exception('Could not parse your date')



    def top_news(self, proxies=None, scraping_bee = None):
        """Return a list of all articles from the main page of Google News
        given a country and a language"""
        d = self.__parse_feed(self.BASE_URL + self.__ceid(), proxies=proxies, scraping_bee=scraping_bee)
        d['entries'] = self.__add_sub_articles(d['entries'])
        return d

    def topic_headlines(self, topic: str, proxies=None, scraping_bee=None):
        """Return a list of all articles from the topic page of Google News
        given a country and a language"""
        #topic = topic.upper()
        if topic.upper() in ['WORLD', 'NATION', 'BUSINESS', 'TECHNOLOGY', 'ENTERTAINMENT', 'SCIENCE', 'SPORTS', 'HEALTH']:
            d = self.__parse_feed(self.BASE_URL + '/headlines/section/topic/{}'.format(topic.upper()) + self.__ceid(), proxies = proxies, scraping_bee=scraping_bee)

        else:
            d = self.__parse_feed(self.BASE_URL + '/topics/{}'.format(topic) + self.__ceid(), proxies = proxies, scraping_bee=scraping_bee)

        d['entries'] = self.__add_sub_articles(d['entries'])
        if len(d['entries']) > 0:
            return d
        else:
            raise Exception('unsupported topic')

    def geo_headlines(self, geo: str, proxies=None, scraping_bee=None):
        """Return a list of all articles about a specific geolocation
        given a country and a language"""
        d = self.__parse_feed(self.BASE_URL + '/headlines/section/geo/{}'.format(geo) + self.__ceid(), proxies = proxies, scraping_bee=scraping_bee)

        d['entries'] = self.__add_sub_articles(d['entries'])
        return d

    def search(self, query: str, helper = True, when = None, from_ = None, to_ = None, proxies=None, scraping_bee=None):
        """
        Return a list of all articles given a full-text search parameter,
        a country and a language
        :param bool helper: When True helps with URL quoting
        :param str when: Sets a time range for the artiles that can be found
        """

        if when:
            query += ' when:' + when

        if from_ and not when:
            from_ = self.__from_to_helper(validate=from_)
            query += ' after:' + from_

        if to_ and not when:
            to_ = self.__from_to_helper(validate=to_)
            query += ' before:' + to_

        if helper == True:
            query = self.__search_helper(query)

        search_ceid = self.__ceid()
        search_ceid = search_ceid.replace('?', '&')

        d = self.__parse_feed(self.BASE_URL + '/search?q={}'.format(query) + search_ceid, proxies = proxies, scraping_bee=scraping_bee)

        d['entries'] = self.__add_sub_articles(d['entries'])
        return d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 2.3 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
# Author - pythoncircle.com

import requests
from bs4 import BeautifulSoup
import json


def print_headlines(response_text):
    soup = BeautifulSoup(response_text, 'lxml')
    headlines = soup.find_all(attrs={"itemprop": "headline"})
    for headline in headlines:
        print("    " + headline.text)


def get_headers():
    return {
        "accept": "*/*",
        "accept-encoding": "gzip, deflate, br",
        "accept-language": "en-IN,en-US;q=0.9,en;q=0.8",
        "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "cookie": "_ga=GA1.2.474379061.1548476083; _gid=GA1.2.251903072.1548476083; __gads=ID=17fd29a6d34048fc:T=1548476085:S=ALNI_MaRiLYBFlMfKNMAtiW0J3b_o0XGxw",
        "origin": "https://inshorts.com",
        "referer": "https://inshorts.com/en/read/",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",
        "x-requested-with": "XMLHttpRequest"
    }

### Parsing

In [ ]:
## Read list of stock tickers
Symbol_list = pd.read_excel('/content/stocks 2.xlsx')

In [ ]:
Symbol_list

,symbol,shortName
0,BTC,Bitcoin


In [ ]:
##Given that parsing was done in steps we manually adjust already parsed stock tickers
Symbol_list.loc[:, 'Parsed']=1
Symbol_list

,symbol,shortName,Parsed
0,BTC,Bitcoin,1


In [ ]:
## init dataset to write to
Bitcoin_news_2014 = pd.DataFrame(list())
Bitcoin_news_2014.to_csv('my_secret_link')
!cp Bitcoin_news_2014.csv "my_secret_link"

cp: cannot stat 'Bitcoin_news_2014.csv': No such file or directory


In [ ]:
News_dataset = pd.read_csv('my_secret_link')

In [ ]:
News_dataset

,Unnamed: 0


In [ ]:
import time
import pandas as pd
import datetime
from tqdm import tqdm


gn = GoogleNews(lang='en')

def get_news(search):
    stories = []
    start_date = datetime.date(2014, 1, 1)
    end_date = datetime.date(2015, 3, 16)
    delta = datetime.timedelta(days=5)
    date_list = pd.date_range(start_date, end_date).tolist()

    for date in tqdm(date_list[:-1], desc=f"Fetching news for {search}"):
        result = gn.search(search, from_=date.strftime('%Y-%m-%d'), to_=(date + delta).strftime('%Y-%m-%d'))
        newsitems = result['entries']

        for item in newsitems:
            story = {
                'title': item['title'],
                'link': item['link'],
                'published': item['published'],
                'source': item['source']
            }
            stories.append(story)

    print("Finished parsing of:", search, "for dates:", start_date, '-', end_date)
    print("Number of news items fetched for", search, ":", len(stories))
    return stories


for index, row in Symbol_list.iterrows():
    if row['Parsed'] == 1:
        print("Parsing news for:", row['shortName'])
        List_of_news = get_news(row['shortName'])
        Data_frac = pd.DataFrame(List_of_news)
        #Data_frac['exchange'] = row['exchange']
        Data_frac['symbol'] = row['symbol']

        with open('my_secret_link', 'a') as f:
          Data_frac.to_csv(f, header=f.tell()==0, index=False)

        print("Parsing news for:", row['symbol'])
        List_of_news2 = get_news(row['symbol'])
        Data_frac2 = pd.DataFrame(List_of_news2)
        #Data_frac2['exchange'] = row['exchange']
        Data_frac2['symbol'] = row['symbol']

        with open('my_secret_link', 'a') as f:
          Data_frac2.to_csv(f, header=False, index=False)


Parsing news for: Bitcoin


Fetching news for Bitcoin: 100%|██████████| 439/439 [04:55<00:00,  1.48it/s]


Finished parsing of: Bitcoin for dates: 2014-01-01 - 2015-03-16
Number of news items fetched for Bitcoin : 12632
Parsing news for: BTC


Fetching news for BTC: 100%|██████████| 439/439 [04:23<00:00,  1.67it/s]

Finished parsing of: BTC for dates: 2014-01-01 - 2015-03-16
Number of news items fetched for BTC : 9165
